<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/omake03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML omake03

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png">


----
# Python + NumPy + Matplotlib + Pandas 入門 (1)
----

次のものの初歩を説明します． ここで「パッケージ」と呼んでいるものは，何か特定の機能を実現するために作られたソフトウェアの集まりを指します．「ライブラリ」や「モジュール」という呼び方をすることもあります．
- NumPy: Python で数値計算できるパッケージ https://numpy.org/
- Matplotlib: Python で様々なグラフを描いたりできるパッケージ https://matplotlib.org/
- Pandas: Python でデータ分析できるパッケージ https://pandas.pydata.org/


----
## NumPy の初歩

NumPy は，Python で数値計算を行うためのパッケージです．ベクトルや行列の形で表される数値データの演算・処理を得意とします．

NumPy を使うためには，次のようにして「`numpy` モジュールをインポート」する必要があります．「モジュール」とか「インポート」とかそういったことはここでは説明しませんので，興味を持ったら自分で調べてみてね．



In [ ]:
import numpy as np  # numpy モジュールをインポートして np という名前で呼べるようにする

自分のPC等に Python 環境を構築している場合は，事前にNumPyパッケージをインストールすることが必要です．
試してみたいひとは takataka に相談するか，自分で調べてみてください．
Google Colab の環境にはあらかじめインストールされているので，インポートするだけで使えます．


`np.array` に数値を並べたリストを渡すと，配列を作ることができます．

In [ ]:
x = np.array([1, 2, 3, 4, 5]) # 整数の配列
print(x)
print(x.shape) # 配列 x の shape（形）．要素数 5 のベクトル（一次元配列）であることがわかる
print(x.dtype) # 配列 x の要素のデータ型．要素は整数型であることがわかる

In [ ]:
y = np.array([1, 2, np.pi, 4.56, 7.89]) # 浮動小数点数の配列． np.pi は円周率
print(y)
print(y.shape)
print(y.dtype)

次の演算が何を行っているか考えよう．

In [ ]:
x2 = x + 2
print(x2)

In [ ]:
z = x + y
print(z)

関数 `np.sum` で要素の和，`np.mean` で平均，`np.var` で分散を求められます．

In [ ]:
print(np.sum(x), np.mean(x), np.var(x))

NumPy には他にも様々な機能がありますが，ここでは説明を省略します．

----
## Matplotlib の初歩

Matplotlib は，Python で様々なグラフを描くことができるパッケージです．NumPy の配列などのデータを可視化するのに便利です．

すごく多機能なんですが，ここでは詳しい使い方は説明せず，ごく簡単な例を示すだけにとどめます．興味を持ったらいろいろ調べてみてください．

In [ ]:
import matplotlib.pyplot as plt # matplotlib の pyplot モジュールを plt という名前でインポート

In [ ]:
X = np.arange(0, 3.0, 0.1)  # 組み込み関数 range の numpy 版
Y = X**2 # 要素ごとに二乗
print(X, Y)

In [ ]:
fig, ax = plt.subplots()  # 描画の準備
ax.plot(X, Y)  # X を X座標の値，Y を Y座標の値として折れ線グラフを描く
plt.show()

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(X, Y, marker='o')
ax.plot(X, np.sqrt(X), color='orange', marker='x')
plt.show()

散布図やヒストグラムを描いたり他にもいろいろできますが，ここでは説明を省略します．

----
## Pandas の初歩

Pandas は，Python で様々なデータ分析を行うことができるパッケージです．

すごく多機能なんですが，以下同文．

In [ ]:
import pandas as pd  # pandas モジュールを pd という名前でインポート

Pandas では， DataFrame （データフレーム）という構造に数値や文字列等様々なデータを格納して処理します．
表計算ソフトのシートのように行・列にデータが並んだものをイメージすればよいでしょう．

In [ ]:
# 3科目の試験の点数5人分
data = [[80,80,80], [82,58,65], [90,90,90], [85,66,63], [49,58,73]]
df = pd.DataFrame(data) # data というリストのリストから DaraFrame を作成
df

In [ ]:
df = pd.DataFrame(data, columns=['数学', '物理', '情報']) # columnsオプションで列名を付けられる
df

様々なデータ分析・統計計算の機能があります．

In [ ]:
df.mean() # 平均

In [ ]:
df.var() # 分散

In [ ]:
df['数学'] # 数学列だけ抽出

In [ ]:
df[df['物理'] >= 60] # 物理が60点以上のひとの点数を抽出

DataFrameは NumPy の配列とみなして計算したり Matplotlib でグラフを描いたり（注）することも可能です．

注: Pandas 自身にも Matplotlib を利用した描画機能がありますが，ここではその説明は省略します

In [ ]:
np.mean(df['数学'])

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 4)) # グラフのサイズを縦横 4 と指定
ax.scatter(df['数学'], df['物理'], marker='o')
ax.set_xlim(0, 100) # X軸の範囲を指定
ax.set_ylim(0, 100) # Y X軸の範囲を指定
ax.set_xlabel('Math') # X 軸のラベル．日本語は文字化けする
ax.set_ylabel('Physics')
plt.show()

Pandas の `read_csv` 関数を使うと，CSVファイル等を読み込んで DataFrame に格納することができます．

In [ ]:
# read_csv の引数に URL を指定すると，ネットワーク越しに CSV ファイル等を読み込むことが可能
dfMPI = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/ex06mpi.csv')
dfMPI

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 4)) # グラフのサイズを縦横 4 と指定
ax.scatter(dfMPI['数学'], dfMPI['物理'], marker='o')
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_xlabel('Math')
ax.set_ylabel('Physics')
plt.show()

----
## 練習問題


(1) 以下に新しいセルを挿入して，dfMPIの「数学」，「物理」，「情報」それぞれの中央値を求めなさい．ただし，上記の説明をふまえて，Pandas で求める方法と NumPy の関数を使って列ごとに求める方法の両方を試すこと．中央値の求め方は調べてね．

(2) 以下に新しいセルを挿入して，dfMPI の

- 「数学」と「物理」
- 「物理」と「情報」

それぞれの散布図を描きなさい．

(3) 以下に新しいセルを挿入して，次のことをやりなさい

1. https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/gorigori.csv
を読み込んで DataFrame を作る
1. その散布図を描く．横軸の範囲は $[-5, 40]$ 縦軸の範囲は $[0, 130]$ としてください．